In [14]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
from glob import glob
import xarray as xr
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from einops import rearrange, reduce
import matplotlib.pyplot as plt
import huggingface_hub as hf_hub
import datasets as ds
from PIL import Image
import torch

from torch.utils.data import DataLoader
# import transforms
from torchvision.transforms import Compose, ToTensor, Normalize

In [15]:
state = "bangladesh"
files = sorted(glob(f"/home/jaiswalsuraj/bkdb/india/punjab/*.zarr"))
print(len(files))

47498


In [16]:
!rm -rf /home/patel_zeel/bk_bd/images
!mkdir /home/patel_zeel/bk_bd/images

def process(file):
    ds = xr.open_zarr(file, consolidated=False)
    val = ds['data'].values
    val = rearrange(val, 'rb cb r c ch -> (rb r) (cb c) ch')
    img = Image.fromarray(val)
    img.save(f"/home/patel_zeel/bk_bd/images/{file.split('/')[-1].replace('.zarr', '.png')}")
    
_ = Parallel(n_jobs=48)(delayed(process)(file) for file in tqdm(files))

  0%|          | 0/47498 [00:00<?, ?it/s]

In [17]:
images = sorted(glob("/home/patel_zeel/bk_bd/images/*.png"))
print(len(images))

chunk_size = 2048
chunks = [images[i:i+chunk_size] for i in range(0, len(images), chunk_size)]

for ci, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    ci = str(ci).zfill(4)
    # create a tar file
    !tar -cf /home/patel_zeel/bk_bd/data/{ci}.tar {" ".join(chunk)}

47498


  0%|          | 0/24 [00:00<?, ?it/s]

tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' 